In [2]:
import numpy as np
import tensorflow as tf
import os
import random 
from collections import defaultdict

In [8]:
# 数据的预处理和加载
def load_data():
    user_ratings = defaultdict(set) #注意这里定义的数据格式，dict[set]
    max_u_id = -1
    max_i_id = -1
    with open('data/u.data', 'r') as f:
        for line in f.readlines():
            u,i,_,_ = line.split("\t")  #这里切分数据
            u = int(u)  #这里将用户编号变换成int格式
            i = int(i)  
            user_ratings[u].add(i)  #这里制作成set形式的[u,i]加入进user_ratings集合
            max_u_id = max(u,max_u_id) #这里的max值即用户的数量，以下同理为商品数
            max_i_id = max(i,max_i_id) 
    
    print("max_u_id:",max_u_id) #这里应该输出943
    print("max_i_idL",max_i_id) #这里应该输出1682
    
    return max_u_id,max_i_id, user_ratings

In [10]:
#  这里对每个用户u，在user_ratings中随机找到一部他评分过的一部电影i,
#  然后保存在user_ratings_test,后面构造训练集和测试集需要用到
#  ?还不是太理解这里的test有什么用
def generate_test(user_ratings):
    user_test = dict()
    for u,i_list in user_ratings.items(): #注意这里循环时，u对应user,i_list对应item集，都是一一对应的
        user_test[u] = random.sample(user_ratings[u],1)[0]
    return user_test

In [14]:
# 训练数据是<u,i,j>的三元组
# 对于随机抽出的用户u，i可以从user_ratings随机抽出，而j也是从总的电影集中随机抽出，
# 当然j必须保证(u,j)不在user_ratings中,也就是用户u未作出反应的电影对象
# 这里采用随机抽样的原因是，论文中提到采用 Bootstrap Sampling的方法
def generate_train_batch(user_ratings, user_ratings_test, item_count, batch_size=512):
    t = []  # 创建训练集
    for b in range(batch_size): #注意这里规定每个训练集大小为512条[u,i,j]
        u = random.sample(user_ratings.keys(),1)[0] 
        i = random.sample(user_ratings[u],1)[0]
        while i == user_ratings_test[u]:
            i = random.sample(user_ratings[u],1)[0]
        
        j = random.randint(1,item_count)
        while j in user_ratings[u]:
            j = random.randint(1,item_count)
        
        t.append([u,i,j])
        
    return np.asarray(t) #调用Numpy库的一个函数，将输入的t转换为浮点数值格式的array

In [15]:
# 对于每个用户u,他的评分电影i是在user_ratings_test中随机抽取的，
# 他的j是用户u所有没有评分过的电影集合
# 比如用户u有1000部电影没有评分，那么这里该用户的测试集样本就有1000个
def generate_test_batch(user_ratings, user_ratings_test, item_count):
    for u in user_ratings.keys():
        t = []
        i = user_ratings_test[u]
        for j in range(1, item_count+1):
            if not(j in user_ratings[u]):
                t.append([u,i,j])
        yield np.asarray(t)             

In [ ]:
    # 论文中提到的参数Theta，其实是用户矩阵和物品矩阵对应的值
    # 对于我们的模型来说，可以简单理解为由id 到embedding的转化
def bpr_mf(user_count, item_count, hidden_dim):
    
    u = tf.placeholder(tf.int32, [None]) #?不知道这里的None指的是什么
    i = tf.placeholder(tf.int32, [None])
    j = tf.placeholder(tf.int32, {None})
    
    # 注意这里的hidden_dim就是我们矩阵分解的隐含维度k
    # user_emb_w对应矩阵W, item_emb_w对应矩阵H
    user_emb_w = tf.get_variable("user_emb_w", [user_count+1, hidden_dim],
                                initializer = tf.random_normal_initializer(0, 0.1)) 
    # 注意这里的初始化tensor生成的是服从正态分布(0,0.1)
    item_emb_w = tf.get_variable("item_emb_w", [item_count+1, hidden_dim],
                                initializer = tf.random_normal_initializer(0, 0.1))
    
    u_emb = tf.nn.embedding_lookup(user_emb_w, u)  # 这里是想用特定u来从大矩阵中检索
    i_emb = tf.nn.embedding_lookup(item_emb_w, i)  # 这里的i也同理
    j_emb = tf.nn.embedding_lookup(item_emb_w, j)  # j同理
    
    # MF predict: u_i > u_j
    # keep_dims=True:按照行的维度求和
    x = tf.reduce_sum(tf.multiply(u_emb, (i_emb-j_emb)),1,keep_dims=True) # ?keep_dims不知道什么意思
    # loss1 = - tf.reduce_mean(tf.log(tf.sigmoid(x)))
    
    # AUC for one user:
    # reasonable if all (u,i,j) pairs are from the same user
    # 
    # average AUC = mean( auc for each user in test set)
    mf_auc = tf.reduce_mean(tf.to_float(x>0))
    
    l2_norm = tf.add_n([
        tf.reduce_sum(tf.multiply(u_emb, u_emb)), #？所以这里为什么要平方再求和
        tf.reduce_sum(tf.multiply(i_emb, i_emb)),
        tf.reduce_sum(tf.multiply(j_emb, j_emb))
    ])
    
    regulation_rate = 0.0001
    bprloss = regulation_rate * l2_norm - tf.reduce_mean(tf.log(tf.sigmoid(x)))
    # ?然后这个损失函数也没看懂。。
    train_op = tf.train.GradientDescentOptimizer(0.01).minimize(bprloss)
    
    return u,i,j,mf_auc,bprloss,train_op


In [ ]:
# 2333代码实在看不懂，我还是继续滚回去看tensorflow吧。
user_count,item_count,user_ratings = load_data()
user_ratins_test = generate_test(user_ratings)

with tf.Session() as sess:
    u,i,j,mf_auc,bprloss,train_op = bpr_mf(user_count, item_count, 20)
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(1,4): # 这里为了测试方便，也就迭代4次
        _batch_bprloss = 0
        for k in range(1,5000):
            uij = generate_train_batch(user_ratings, user_ratings_test, item_count)
            _bprloss, _train_op  = sees.run([bprloss,train_op],
                                           feed_dict={u:uij[:,0],i:uij[:,j],j:uij[:,2]})
            _batch_bprloss += _bprloss
            
            print("epoch:", epoch)
            print("bpr_loss:", _batch_bprloss / k)
            print("_train_op")
            
            user_count = 0
            _auc_sum = 0.0
            
            for t_uij in generate_train_batch(user_ratings, user_ratings_test, item_count):
                _auc, _test_bprloss = sess.run([mf_auc, bprloss],
                                              feed_dict={u:t_uij[:,0],i:t_uij[:,1],j:t_uij[:,2]})
            